In [ ]:
import os
os.environ["OMP_NUM_THREADS"] = "16"

In [ ]:
import healpy as hp
import numpy as np

https://wiki.cosmos.esa.int/planck-legacy-archive/index.php/Map-making#HFI_2018_Solar_dipole

HFI solar dipole 2018

v = (369.8150 ± 0.0010) km s-1

A = (3362.08 ± 0.09 (stat.) ± 0.45 (syst.) ± 0.32 (cal.) μK

l = (264.021 ± 0.003 (stat.) ± 0.0079 (syst.) )°

b = (48.253 ± 0.001 (stat.) ± 0.0037 (syst.) )°



In [ ]:
nside = 512
npix = hp.nside2npix(nside)

In [ ]:
dip_lon = 264.021
dip_lat = 48.253

In [ ]:
import pysm3.units as u
from astropy.constants import c

In [ ]:
c

In [ ]:
T_CMB = 2.72548  * u.K_CMB
vel = 369815 * (u.m/u.s)
beta = vel / c
gamma=1/np.sqrt(1-beta**2)
cosdir = 1
T_dipole_CMB = T_CMB / (gamma * ( 1 - beta * cosdir ))
solar_dipole_amplitude = T_dipole_CMB - T_CMB

In [ ]:
solar_dipole_amplitude.to(u.mK_CMB)

In [ ]:
gal2eq = hp.Rotator(coord="GC")

In [ ]:
solar_dipole_vec_gal = gal2eq(hp.ang2vec(dip_lon, dip_lat, lonlat=True))

In [ ]:
solar_dipole_vec_gal

In [ ]:
vec = np.array(hp.pix2vec(nside, np.arange(npix)))

In [ ]:
vec.shape

In [ ]:
cosdir = np.dot(solar_dipole_vec_gal, vec)

In [ ]:
solar_dipole_map = T_CMB / (gamma * ( 1 - beta * cosdir )) - T_CMB

In [ ]:
cosdir.max()

In [ ]:
%matplotlib inline

In [ ]:
hp.mollview(solar_dipole_map)

In [ ]:
hp.mollview(solar_dipole_map, coord="CG")

In [ ]:
solar_dipole_map

In [ ]:
unit = "uK_CMB"

In [ ]:
!mkdir -p ../output/dipole/

In [ ]:
tag = "dipole"
pixelization = "healpix"

In [ ]:
hp.write_map(
    f"../output/dipole/sobs_mbs-s0016-20241111_{telescope}_mission_{band}_{tag}_{pixelization}.fits",
    hp.reorder(solar_dipole_map.to_value(unit), r2n=True),
    coord="C",
    column_units=unit,
    dtype=np.float32,
    overwrite=True,
    nest=True,
)